In [1]:
!pip install nvidia-riva-client
!pip install numpy
!pip install tritonclient[all]

  Using cached grpcio-1.50.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
  Using cached protobuf-4.21.9-cp37-abi3-manylinux2014_x86_64.whl (408 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached grpcio-1.41.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.9
    Uninstalling protobuf-4.21.9:
      Successfully uninstalled protobuf-4.21.9
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.50.0
    Uninstalling grpcio-1.50.0:
      Successfully uninstalled grpcio-1.5

In [2]:
import riva.client
auth = riva.client.Auth(uri='10.100.182.246:50051')

In [3]:
import numpy as np
import tritonclient.http as httpclient

from tritonclient.utils import np_to_triton_dtype

URL = "10.98.96.143:8000"
MODEl_GPTJ_FASTERTRANSFORMER = "ensemble" 

OUTPUT_LEN = 128
BATCH_SIZE = 1
BEAM_WIDTH = 1
TOP_K = 1
TOP_P = 0.0

start_id = 220
end_id = 50256

In [4]:
client = httpclient.InferenceServerClient(URL,
                                           concurrency=1,
                                           verbose=False)


In [5]:
client.get_server_metadata()

{'name': 'triton',
 'version': '2.18.0',
 'extensions': ['classification',
  'sequence',
  'model_repository',
  'model_repository(unload_dependents)',
  'schedule_policy',
  'model_configuration',
  'system_shared_memory',
  'cuda_shared_memory',
  'binary_tensor_data',
  'statistics']}

In [6]:
# Inference hyperparameters
def prepare_tensor(name, input):
    tensor = httpclient.InferInput(
        name, input.shape, np_to_triton_dtype(input.dtype))
    tensor.set_data_from_numpy(input)
    return tensor

# explanation
def prepare_inputs(input0):
    bad_words_list = np.array([[""]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    input0_data = np.array(input0).astype(object)
    output0_len = np.ones_like(input0).astype(np.uint32) * OUTPUT_LEN
    runtime_top_k = (TOP_K * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    runtime_top_p = TOP_P * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    beam_search_diversity_rate = 0.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    temperature = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    len_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    repetition_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    random_seed = 0 * np.ones([input0_data.shape[0], 1]).astype(np.int32)
    is_return_log_probs = True * np.ones([input0_data.shape[0], 1]).astype(bool)
    beam_width = (BEAM_WIDTH * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    start_ids = start_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)
    end_ids = end_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)

    inputs = [
        prepare_tensor("INPUT_0", input0_data),
        prepare_tensor("INPUT_1", output0_len),
        prepare_tensor("INPUT_2", bad_words_list),
        prepare_tensor("INPUT_3", stop_words_list),
        prepare_tensor("runtime_top_k", runtime_top_k),
        prepare_tensor("runtime_top_p", runtime_top_p),
        prepare_tensor("beam_search_diversity_rate", beam_search_diversity_rate),
        prepare_tensor("temperature", temperature),
        prepare_tensor("len_penalty", len_penalty),
        prepare_tensor("repetition_penalty", repetition_penalty),
        prepare_tensor("random_seed", random_seed),
        prepare_tensor("is_return_log_probs", is_return_log_probs),
        prepare_tensor("beam_width", beam_width),
        prepare_tensor("start_id", start_ids),
        prepare_tensor("end_id", end_ids),
    ]
    return inputs

In [7]:
input_user = (
    "Polish: Gdzie jest najbliższa stacja kolejowa? \n"
    "English: Where is the nearest train station? \n\n"
    "Polish: Ile kosztuje bilet kolejowy do Warszawy? \n"
    "English: How much is a train ticket to Warsaw? \n\n"
    "Polish: Niestety nie mówię po Francusku. \n"
    "English: ")

In [8]:
input_user = (
    "French: Il n'est guère nécessaire d'insister sur l'importance de l'étude de "
"Mythologie : nos poèmes, nos romans et même nos journaux quotidiens regorgent de "
"allusions classiques; une visite dans nos galeries d'art et nos musées ne peut pas non plus être "
"pleinement apprécié sans quelque chose de plus qu'une simple connaissance superficielle d'un "
"sujet qui a de tout temps inspiré peintres, sculpteurs et poètes. Ce "
"il ne me reste donc plus qu'à exprimer l'espoir que mon petit travail pourra "
"s'avérer utile, non seulement pour les enseignants et les universitaires, mais aussi pour une classe nombreuse "
"de lecteurs généraux, qui, en passant une heure de loisir, peuvent en tirer "
"plaisir et profiter de sa lecture. \n"
"English: It is hardly necessary to dwell upon the importance of the study of "
"Mythology: our poems, our novels, and even our daily journals teem with "
"classical allusions; nor can a visit to our art galleries and museums be "
"fully enjoyed without something more than a mere superficial knowledge of a "
"subject which has in all ages inspired painters, sculptors, and poets. It "
"therefore only remains for me to express a hope that my little work may "
"prove useful, not only to teachers and scholars, but also to a large class "
"of general readers, who, in whiling away a leisure hour, may derive some "
"pleasure and profit from its perusal. \n\n"
"French: J'ajoute qu'aucun effort n'a été épargné pour que, sans passer "
"sur des détails dont l'omission aurait {ii} entaché l'exhaustivité "
"de l'œuvre, pas un seul passage ne doit être trouvé qui pourrait éventuellement "
"offenser la délicatesse la plus scrupuleuse; et aussi que j'ai volontairement traité "
"le sujet avec cette révérence que je considère due à tout religieux "
"système, même erroné. \n"
"English: I may add that no pains have been spared in order that, without passing "
"over details the omission of which would have {ii} marred the completeness "
"of the work, not a single passage should be found which could possibly "
"offend the most scrupulous delicacy; and also that I have purposely treated "
"the subject with that reverence which I consider due to every religious "
"system, however erroneous. \n\n"
"French:  En m'efforçant de répondre à ce besoin, j'ai cherché à placer devant le "
"lecteur une image réaliste des divinités de l'époque classique telles qu'elles étaient "
"conçu et adoré par les anciens eux-mêmes, et ainsi réveiller"
"dans l'esprit des jeunes étudiants un désir de devenir plus intime"
"connaître les nobles productions de l'antiquité classique.\n"
"English: "
)

In [17]:
input_user = (
        "Before entering upon the many strange beliefs of the ancient Greeks, and "
"the extraordinary number of gods they worshipped, we must first consider "
"what kind of beings these divinities were. \n "
    " Prima di addentrarci nelle tante strane credenze degli antichi greci, e "
"lo straordinario numero di dei che adoravano, dobbiamo prima considerare "
"che tipo di esseri erano queste divinità. \n\n"
        "They were also more "
"commanding in stature, height being considered by the Greeks an attribute "
"of beauty in man or woman. \n "
    "Erano anche di più "
"di statura imponente, essendo l'altezza considerata dai Greci un attributo "
"di bellezza nell'uomo o nella donna. \n\n"
    "The Greeks believed that the mental qualifications of their gods were of a "
"much higher order than those of men. \n"
    ""
)

In [64]:
input0 = [[input_user],]
inputs = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
output0 = result.as_numpy("OUTPUT_0")

In [65]:
print(output0[0].decode('UTF-8'))

English: Before entering upon the many strange beliefs of the ancient Greeks, and the extraordinary number of gods they worshipped, we must first consider what kind of beings these divinities were. 
 Italian: Prima di addentrarci nelle tante strane credenze degli antichi greci, e lo straordinario numero di dei che adoravano, dobbiamo prima considerare che tipo di esseri erano queste divinità. 

English: They were also more commanding in stature, height being considered by the Greeks an attribute of beauty in man or woman. 
 Italian: Erano anche di più di statura imponente, essendo l'altezza considerata dai Greci un attributo di bellezza nell'uomo o nella donna. 

English: The Greeks believed that the mental qualifications of their gods were of a much higher order than those of men. 
Italian: 
 The Greeks believed that the mental qualifications of their gods were of a much higher order than those of men. 
 Italian: I Greci credevano che le qualificazioni mentali delle loro divinità foss

In [9]:
language_code = 'en-US'
sample_rate_hz = 16000
nchannels = 1
sampwidth = 2
text = (
)

In [18]:
input0 = [[input_user],]
inputs0 = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs0)
output0 = result.as_numpy("OUTPUT_0")

TOP_K = 0.3
TOP_P = 0.8
for p in input_user.split("\n\n"):
    input1 = [[p],]
    inputs1 = prepare_inputs(input1)
    result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs1)
    output1 = result.as_numpy("OUTPUT_0")
    strOut = output0[0].decode('UTF-8').split("\n\n")
    print(strOut[1], "\n")



They were also more commanding in stature, height being considered by the Greeks an attribute of beauty in man or woman. 
 Erano anche di più di statura imponente, essendo l'altezza considerata dai Greci un attributo di bellezza nell'uomo o nella donna.  

They were also more commanding in stature, height being considered by the Greeks an attribute of beauty in man or woman. 
 Erano anche di più di statura imponente, essendo l'altezza considerata dai Greci un attributo di bellezza nell'uomo o nella donna.  

They were also more commanding in stature, height being considered by the Greeks an attribute of beauty in man or woman. 
 Erano anche di più di statura imponente, essendo l'altezza considerata dai Greci un attributo di bellezza nell'uomo o nella donna.  

